In [1]:
import xarray as xr
from datetime import datetime
import numpy as np
from datetime import timedelta
from climpy.hazard.condition import MaskArea, ThresholdQuantile, ConnectStructure, AverageArea
from climpy.hazard.hazard import PointHazard
from climpy.hazard.select import EventSelector, TimeBetweenEvents
from climpy.hazard.criterion import ArialCriterion
from climpy.hazard.link import ValueOverTimeInterval, ValueAtTimeOffset

import matplotlib.pyplot as plt

In [2]:
catch_id = 6337060

shp_path = f"/data/compoundx/GRDC_runoff/Xie_bnd/individual_shp/{catch_id}.geojson"
input_dir = '/data/compoundx/causal_flood/grdc/GRDC-Daily.nc'
runoff = xr.open_dataset(input_dir)["runoff_mean"].sel(id = catch_id)
met_path = f"/data/compoundx/causal_flood/basins_gridded/observations_meteorological/{catch_id}/"
tavg_data = xr.open_dataset(met_path + "tavg.nc")["tavg"]
pre_data = xr.open_dataset(met_path + "pre.nc")["pre"]

In [3]:
quantile = 0.90
structure = np.array([1,1,1])

flood_conditions = [
    ThresholdQuantile('>', quantile),
    ConnectStructure(structure)
]

In [4]:
flood = PointHazard(flood_conditions)
flood.apply_conditions(runoff)

In [5]:
event_list = flood.all_events()
len(event_list)

317

In [6]:
event_selected_list = EventSelector(event_list).filter_by_rule(TimeBetweenEvents( '>', timedelta(days=10)))
len(event_selected_list)

265

In [7]:
preprocessing_conditions = ArialCriterion([
    MaskArea(shp_path=shp_path),
    AverageArea(),
])

In [8]:
tavg_processed = preprocessing_conditions.apply_conditions(tavg_data)
pre_processed = preprocessing_conditions.apply_conditions(pre_data)

In [13]:
link = ValueAtTimeOffset(timedelta(days=-6))
link.on_events(event_list=event_selected_list)
t_avg_7day_before = link.get_values(tavg_processed)

In [14]:
link = ValueOverTimeInterval(timedelta(days=-6))
link.on_events(event_list=event_selected_list)
pre_last_7days = link.get_values(pre_processed)